In [1]:
import pandas as pd
import os, json
import requests

Import and clean the data

In [2]:
name_list=os.listdir('raw_data/json')

In [14]:
def get_data_from_json(file_name):
    """
    Loads a json file from a set location and returns a Pandas dataframe with its items data.
    If the file has no item inputs the function returns None.
    """
    json_file = 'raw_data/json/' + file_name
    with open(json_file,'r') as f:
        all_data = json.load(f)
        data = all_data['items'] 
        if data == []:
            return None
        df = pd.DataFrame.from_records(data)[['id','word','rate', 'username', 'sex', 'code',
                                          'country','num_votes', 'num_positive_votes', 'pathmp3']]
    return df

In [5]:
def filter_and_merge_json(name_list):
    """
    Loops through json files in a list and adds the first 2 item inputs from each to a dataframe.
    If there is only one input, it is added to a seperate dataframe. 
    """
    df_list=[]
    df_list_one_mp3=[]
    for file in name_list:
        df=get_data_from_json(file)
    
    #get top two results
        df=df.head(2)
        if df.shape[0] == 2:
        
    #create lists of dataframes
            df_list.append(df)
        elif df.shape[0] == 1:
            df_list_one_mp3.append(df)
    
    #merge dataframes
    main_df = pd.concat(df_list)
    main_df=main_df.reset_index(drop=True)
    if len(df_list_one_mp3)!=0:
        single_mp3_df = pd.concat(df_list_one_mp3)
    return main_df, single_mp3_df

In [6]:
main_df, single_mp3_df = filter_and_merge_json(name_list)

Download the MP3 files

In [16]:
def save_mp3(df, two_inputs=True):
    """
    Uses a dataframe with words and a link to an their pronunciation audio and saves the audio inputs as mp3 files.
    two_inputs == True: the dataframe has 2 instances of each word and the mp3 file of each is saved in a different directory.
    
    """
    if two_inputs==True:
        word=''
        for index,row in df.iterrows():
            response = requests.get(row['pathmp3'])
            name=row['word']
            if word != name:
                with open(f'api_data/speaker1/{name}.mp3','wb') as f:
                    f.write(response.content)
                    word=name
            else:
                 with open(f'api_data/speaker2/{name}.mp3','wb') as f:
                    f.write(response.content) 
        return 
    #if df has one input per word
    else:
        for index,row in df.iterrows():
            response = requests.get(row['pathmp3'])
            name=row['word']
            with open(f'api_data/single_speaker/{name}.mp3','wb') as f:
                f.write(response.content)
        return

In [46]:
save_mp3(main_df)

In [ ]:
save_mp3(single_mp3_df, two_inputs = False)